In [ ]:
import folium
from ipyleaflet import Map, Marker
from folium.plugins import MarkerCluster
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import geojson
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from branca.element import Figure
import branca.colormap as cm
from folium.plugins import TimeSliderChoropleth
from datetime import datetime
from tqdm import tqdm
import seaborn as sns
from folium.plugins import TimestampedGeoJson
import warnings
warnings.filterwarnings('ignore')
from tabulate import tabulate
from sqlalchemy import create_engine
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>pre { white-space: pre !important; }</style>"))

# Time Slider for LEA Map

In [ ]:
gpd_covid_lea_map = gpd.read_file("./DCU Data/COVID-19_HPSC_HIU_Timeseries_Local_Electoral_Area_Mapped/COVID-19_HPSC_HIU_Timeseries_Local_Electoral_Area_Mapped.shp")

In [ ]:
gpd_covid_lea_map.head()

In [ ]:
len(gpd_covid_lea_map["FixedDATE"].unique().tolist())

### Clean Covid Data

In [ ]:
def db_connection():
    config = {
       'host': "upcom.mysql.database.azure.com",
       'user': "covid@upcom",
       'password': "VaL0S8^od",
       'database': "upcom",
    }
    engine = create_engine(
       "mysql+pymysql://" + config["user"] + ":" + config["password"] + "@" + config["host"] + "/" + config[
           "database"], connect_args={'ssl': {'fake_flag_to_enable_tls': True}})
    return engine

def get_chunks_df(le_incidence,have_0_index_ls):
    lst_df = []
    block_df = le_incidence.loc[:have_0_index_ls[0]]
    if block_df.shape[0] > 1:
        lst_df.append(block_df)
    i = 0
    while i < len(have_0_index_ls)-1:
        block_df = le_incidence.loc[have_0_index_ls[i]:have_0_index_ls[i+1]-1]
        lst_df.append(block_df)
        i+=1
    block_df = le_incidence.loc[have_0_index_ls[-1]:]
    if block_df.shape[0] > 0:
        lst_df.append(block_df)
    return lst_df

def cal_p7_upward(le_incidence,start_index,first_p7,max_possi):
    le_incidence.loc[start_index - 1, 'P7'] = max_possi - first_p7
    temp_up = le_incidence.loc[:start_index].iloc[::-1]
    for i in range(1, len(temp_up.index)):
        temp_up.loc[temp_up.index.tolist()[1] - i, "P7"] = temp_up["P14"].values[i]-temp_up["P7"].values[i]
    temp_up = temp_up.iloc[::-1][1:]
    return temp_up

def cal_p7_downward(le_incidence,max_possi,iszero):
    start_index =le_incidence.index[0]
    temp_down = le_incidence
    for i in range(start_index + 1, temp_down.index[-1] + 1):
        if (temp_down.loc[i,"P14"]!=0):
            temp_down.loc[i, "P7"] = temp_down.loc[i, "P14"] - temp_down.loc[i - 1, "P7"]
            if ((temp_down.loc[i, "P14"] - temp_down.loc[i - 1, "P7"]) < 0):
                temp_down.loc[i, "P7"] = 0
        else:
            temp_down.loc[i, "P7"] = max_possi - temp_down.loc[i - 1, "P7"]
    return temp_down

def change_zero_two(ls):
    chosen_ls = []
    for i in range(1,len(ls)-1):
        if (ls[i]-ls[i-1]==1) and (ls[i+1]-ls[i]==1):
            chosen_ls.append(ls[i])
    return chosen_ls

def pick_case(le_incidence,lst_possibility,dict_possible,le_no_zero,rightous_df,have_0):
    if len(lst_possibility)==1:
        rightous_df.append(le_incidence["LE_ID"].unique().tolist()[0])
        le_incidence.update(lst_possibility[0])
    elif len(lst_possibility)>1:
        chosen_one = lst_possibility[0]
        dict_possible[le_incidence["LE_ID"].unique().tolist()[0]]=lst_possibility
        le_incidence.update(chosen_one)
    else:
        le_incidence["P7"]=None
        le_no_zero.append(le_incidence["LE_ID"].unique().tolist()[0])
    return le_incidence

def find_appropriate_cases_initial(le_id,have_0,le_incidence,cleaned_county_p7,iszero):
    possible_result_ls = []
    max_poss=4
    # Block of DataFrame
    if iszero:
        have_0_index_ls = have_0.index.tolist()
        chosen_starting = have_0_index_ls[0]
    else:
        chosen_starting = le_incidence.index.tolist()[0]
    df_up = le_incidence.loc[:chosen_starting]
    df_down = le_incidence.loc[chosen_starting:]
        
    for possi_initial_p7 in range(max_poss+1):
        df_down.loc[df_down.index[0], "P7"] = possi_initial_p7
        # Calculate P7_100k downward
        temp_down = cal_p7_downward(df_down,max_poss,iszero)
        if (temp_down["P7"]>=0).all():
            # Calculate P7_100k upward
            temp_up = cal_p7_upward(df_up,chosen_starting,possi_initial_p7,max_poss)[:-1]
            if (temp_up["P7"]>=0).all():
                temp_df = pd.concat([temp_up, temp_down])
                possible_result_ls.append(temp_df)
    return possible_result_ls

def generate_p7_100k(le_id,have_0,le_incidence,dict_possible,le_no_zero,rightous_df,cleaned_county_p7_p14,iszero):
    lst_possibility = find_appropriate_cases_initial(le_id,have_0,le_incidence,cleaned_county_p7_p14,iszero)
    le_incidence = pick_case(le_incidence,lst_possibility,dict_possible,le_no_zero,rightous_df,have_0)
    return le_incidence

def get_county_p7(df,days):
    df["TimeStamp"] = pd.to_datetime(df["TimeStamp"])
    if days == 7:
        new_df = df[(df["TimeStamp"]>=datetime.strptime("2020/08/02",'%Y/%m/%d')) & (df["TimeStamp"]<=datetime.strptime("2021/11/15",'%Y/%m/%d')) ].sort_values(by=["CountyName","TimeStamp"])
    else:
        new_df = df[(df["TimeStamp"]>=datetime.strptime("2020/07/26",'%Y/%m/%d')) & (df["TimeStamp"]<=datetime.strptime("2021/11/15",'%Y/%m/%d')) ].sort_values(by=["CountyName","TimeStamp"])
    ls_df = []
    for county in new_df["CountyName"].unique().tolist():
        county_df = new_df[new_df["CountyName"]==county].reset_index(drop=True)
        county_df["Actual_New_Cases"]=county_df["ConfirmedCovidCases"]-county_df["ConfirmedCovidCases"].shift(1)
        county_df.drop([0],inplace=True)
        county_df.drop(columns=["ConfirmedCovidCases"],inplace=True)
        county_df.reset_index(drop=True,inplace=True)
        county_df.drop([0], inplace=True)
        if days==7:
            date_p7 = [county_df.loc[i + 1, "TimeStamp"] for i in county_df.index.tolist() if
                       (i + 1) % (days) == 0]
            p7_cases = [county_df.loc[i-days+1:i]["Actual_New_Cases"].sum() for i in county_df.index.tolist() if (i)%days==0]
            new_county_df = pd.DataFrame({"County":[county for i in range(68)],"Date":date_p7,"P7":p7_cases})
        else:
            date_p7 = [county_df.loc[i + 1, "TimeStamp"] for i in county_df.index.tolist() if (i + 1) % (days//2) == 0][1:]
            p7_cases = [county_df.loc[i-days+1:i]["Actual_New_Cases"].sum() for i in county_df.index.tolist() if (i%(days//2)==0)][1:]
            new_county_df = pd.DataFrame({"County":[county for i in range(68)],"Date":date_p7,"P14":p7_cases})
        ls_df.append(new_county_df)
    result = pd.concat(ls_df).reset_index(drop=True)
    result["County"] = result["County"].str.upper()
    return result

def find_correct_p14(row):
    accepted_index = [name for name in row.index.tolist() if ("P7" in name) & (name!="P7_County")]
    needed_row = row[accepted_index]
    if (needed_row[needed_row.isnull()].shape[0])==1:
        replaced_value = (row["P7_County"]-needed_row[~needed_row.isnull()].sum())//needed_row[needed_row.isnull()].shape[0]
        if replaced_value >0:
            needed_row[needed_row.isnull()] = replaced_value
        else:
            needed_row[needed_row.isnull()]= 0
        row.update(needed_row)
    else:
        handling_le_cases = [i[3:] for i in row[row.isnull()].index.tolist()]
        for le_id in handling_le_cases:
            row["P7_"+le_id] = row["P14_"+le_id]/row["P14_County"]*row["P7_County"]
    return row

def check_if_exist(le1,le2):
    for i in le1:
        if i in le2:
            return True
    return False

def find_right_cases_initial(county_df,county_p7_p14,le_no_zero):
    new_index_county = county_df.sort_values(by=["LE_ID","FixedDATE"]).reset_index(drop=True)
    sub_lea_ls = [new_index_county.loc[i - 66:i].reset_index(drop=True)[["LE_ID","FixedDATE","P14","P7"]] for i in new_index_county.index.tolist() if (i+1) % 67 == 0]
    have_0_df_ls = sorted(set(list(sum([df[df["P14"]==0].index.tolist() for df in sub_lea_ls],[]))))
    prv_curr_0_df_ls = sorted(list(set(sum([[i-1,i] for i in have_0_df_ls if (i>0)],[]))))
    if have_0_df_ls[0]==0:
        prv_curr_0_df_ls=[0]+prv_curr_0_df_ls
    lea_df = pd.concat(sub_lea_ls,axis=1)
    lea_df["Date"]=lea_df["FixedDATE"].T.drop_duplicates().T
    lea_df.drop(columns=["FixedDATE"],inplace=True)
    county_incidence = county_p7_p14[county_p7_p14["County"]==county_df["COUNTY"].unique().tolist()[0]]
    county_incidence.rename(columns={"P14":"P14_County","P7":"P7_County"},inplace=True)
    col_ls = lea_df.columns.tolist()
    for index,col in enumerate(col_ls):
        if col=="P14":
            col_ls[index] = col+"_"+str(float(lea_df.iloc[:,(index-1)].unique()[0]))
        elif col=="P7":
            col_ls[index] = col + "_" + str(float(lea_df.iloc[:, (index - 2)].unique()[0]))
    lea_df.columns = col_ls
    le_ls = county_df["LE_ID"].unique().tolist()
    county_incidence["Date"]=county_incidence["Date"].astype(str)
    merged_county_lea = lea_df.merge(county_incidence.set_index("Date"), left_on="Date", right_index=True)
    if (check_if_exist(le_ls,le_no_zero)):
        lea_df.drop(columns=["LE_ID"],inplace=True)
        merged_county_lea["Current Total"]=merged_county_lea[[col for col in merged_county_lea.columns if ("P7" in col) and ("County" not in col)]].sum(axis=1)
        p14ls = [col for col in merged_county_lea.columns.tolist() if ("P7" in col) | ("P14" in col)]
        p14_df = merged_county_lea[p14ls]
        p14_df = p14_df.apply(lambda x: find_correct_p14(x),axis=1)
        merged_county_lea.update(p14_df)
        county_df.update(update_initial_record(merged_county_lea,county_df,le_no_zero))
    return county_df

def update_initial_record(merged_county_lea,reorder_incidence,le_no_zero):
    le_ls = [float(i) for i in merged_county_lea["LE_ID"].drop_duplicates().values.tolist()[0]]
    replaced_le_id = list(set(le_no_zero).intersection(set(le_ls)))
    temp_merged_lea = merged_county_lea.drop(columns=["LE_ID"])
    none_county = reorder_incidence[reorder_incidence["P7"].isnull()&reorder_incidence["LE_ID"].isin(replaced_le_id)]
    for le_id in replaced_le_id:
        temp_le_df = none_county[none_county["LE_ID"]==le_id]
        temp_le_df["P7"] = temp_merged_lea["P7_"+str(float(le_id))].values
        none_county.update(temp_le_df)
    reorder_incidence.update(none_county)
    return reorder_incidence

def find_p7_ireland(gpd_covid_lea_map):
    engine = db_connection()    
    incidence_df = pd.DataFrame(gpd_covid_lea_map.drop(columns=["geometry","FID","DATE","GUID"]))
    demo_df = pd.read_sql("SELECT LE_ID,total_population_age FROM upcom.lea_details",con=engine).drop_duplicates()
    demo_df["LE_ID"] = demo_df["LE_ID"].astype(str)
    incidence_df = pd.merge(incidence_df,demo_df,on="LE_ID")
    reorder_incidence = incidence_df.sort_values(by=["LE_ID", "FixedDATE"]).reset_index(drop=True)
    reorder_incidence["P14"]=reorder_incidence["Rate100K"]*reorder_incidence["total_population_age"]/100000
    reorder_incidence["P14"] = round(reorder_incidence["P14"])
    reorder_incidence["P7"] = None
    reorder_incidence["P7_100k"] = None
    le_no_zero = []
    dict_possible= {}
    rightous_df_ls = []
    le_lst = reorder_incidence["LE_ID"].unique().tolist()
    county_cases = pd.read_csv("./DCU Data/COVID-19_HPSC_County_Statistics_Historic_Data.csv",usecols=["CountyName","ConfirmedCovidCases","TimeStamp"])
    county_cases["TimeStamp"]=county_cases["TimeStamp"].str.split(" ").str[0]
    cleaned_county_p7 = get_county_p7(county_cases,7)
    cleaned_county_p14 = get_county_p7(county_cases,14)
    cleaned_county_p7_14 = pd.concat([cleaned_county_p7,cleaned_county_p14[["P14"]]],axis=1)


    # Find possible P7
    for index in tqdm(range(len(le_lst))):
        le_id = le_lst[index]
        le_incidence = reorder_incidence[reorder_incidence["LE_ID"] == le_id]
        have_0 = le_incidence[le_incidence["Rate100K"] == 0]
        iszero=True
        if (have_0.shape[0] == 0):
            iszero=False
        le_incidence = generate_p7_100k(le_id,have_0,le_incidence,dict_possible,le_no_zero,rightous_df_ls,cleaned_county_p7_14,iszero)
        reorder_incidence.update(le_incidence)

    reorder_incidence["P7_100k"]=reorder_incidence["P7"]*100000/reorder_incidence["total_population_age"]
    return reorder_incidence


In [ ]:
covid_data = find_p7_ireland(gpd_covid_lea_map)

In [ ]:
print(tabulate(covid_data.head(), headers='keys', tablefmt='psql'))

### Clean Covid map

In [ ]:
# Join cleaned COvid data with Geometry
gpd_covid_lea_map[["P14","P7","P7_100k"]] = covid_data[["P14","P7","P7_100k"]]
# Prepare data
max_colour = max(gpd_covid_lea_map['Rate100K'])
min_colour = sorted(gpd_covid_lea_map['Rate100K'].unique().tolist())[1]
defined_scale = [i for i in range(int(min_colour), int(max_colour)+int(max_colour//12), int(max_colour//12))]
cmap = cm.linear.RdYlBu_11.to_step(12,index=defined_scale)
cmap.caption = "Number of COVID cases by LEAs every week"
gpd_covid_lea_map['colour'] = gpd_covid_lea_map['Rate100K'].map(cmap)
gpd_covid_lea_map_only_0 = gpd_covid_lea_map[gpd_covid_lea_map["Rate100K"]==0]
gpd_covid_lea_map_only_0["colour"]=""
gpd_covid_lea_map.update(gpd_covid_lea_map_only_0)
gpd_covid_lea_map["DATE"]=(pd.DatetimeIndex(gpd_covid_lea_map['FixedDATE']).astype(int)// 10 ** 9).astype('U10')
temp_gpd = gpd_covid_lea_map[gpd_covid_lea_map["colour"]==""]
temp_gpd["colour"]="#00FFFFFF"
gpd_covid_lea_map.update(temp_gpd)
short_gpd = gpd_covid_lea_map[["LE_ID","geometry"]].drop_duplicates().reset_index()
short_gpd.drop(columns=["index"],inplace=True)
lea_list = short_gpd['LE_ID'].unique().tolist()
lea_idx = range(len(lea_list))
style_dict = {}
for i in lea_idx:
    lea = lea_list[i]
    result = gpd_covid_lea_map[gpd_covid_lea_map['LE_ID'] == lea]
    inner_dict = {}
    for _, r in result.iterrows():
        inner_dict[r['DATE']] = {'color': r['colour'], 'opacity': 1}
    style_dict[i] = inner_dict

new_lea = gpd_covid_lea_map.drop(columns=["Rate100K","DATE"]).drop_duplicates()
new_lea = new_lea.reset_index().drop(columns=["index"])
    

### Generate Map

In [ ]:
# Generate Map
fig6=Figure(height=850,width=1000)
center = [53.52876390714696, -8.070723942287891]
bounds = [[51.381808536222344, -4.9184898320881025],[55.45217379512414, -11.451803787292237]]
slider_map = folium.Map(location = center, zoom_start=7, tiles='cartodbpositron',min_zoom = 7,maxBounds= bounds,
  maxBoundsViscosity= 1)


# lea_layer = folium.FeatureGroup(name="LEAs",show=True).add_to(slider_map)
# style2 = {'fillColor': '#00FFFFFF', 'color': '#00FFFFFF'}
# lea_layer.add_child(folium.GeoJson(data=short_gpd,popup=folium.GeoJsonPopup(fields=["LE_ID"]),style_function=lambda x:style2)).add_to(slider_map)



fig6.add_child(slider_map)
g = TimeSliderChoropleth(
    short_gpd.to_json(),
    styledict=style_dict,
).add_to(slider_map)
g = cmap.add_to(slider_map)
slider_map.save("TimeSlice.html")

# Road network and traffic counter

In [ ]:
gpd_network = gpd.read_file("./DCU Data/Roads_-_OSi_National_250k_Map_of_Ireland/Roads_-_OSi_National_250k_Map_of_Ireland.shp")
traffic_location_df = pd.read_csv("./DCU Data/tmu-sites.tsv",sep='\t',header=None)
traffic_count_df = pd.read_csv("traffic_count_week.csv")
traffic_count_df["siteId"] = traffic_count_df["siteId"].astype(str)

In [ ]:
for i in traffic_count_df["siteId"].unique():
#     print(tabulate(traffic_count_df.head(100), headers='keys', tablefmt='psql'))
    print (len((traffic_count_df[traffic_count_df["siteId"]==i]["Time"].unique().tolist())))

### Clean Traffic Counter

In [ ]:
# Clean traffic_location_df
# Generate map
def generate_map(map_mpp,df,column_to_show,color,marker_cluster):
    type_data = "Traffic Counter"
    show_type=True
    locations = df[['Lat', 'Lon']]
    locationlist = locations.values.tolist()
    for point in range(0, len(locationlist)):
        folium.Marker(locationlist[point], popup=folium.Popup(folium.IFrame("<b>Name:</b> "+df[column_to_show][point]+'<br><b>Type:</b> '+type_data,width=250,height=80),max_width=250), icon = folium.Icon(color=color)).add_to(marker_cluster)
   
    return map_mpp

# Convert Lat Lon columns to Geometry column
def generate_geometry(df,lat,lon):
    geometry = [Point(xy) for xy in zip(df[lon], df[lat])]
    df = gpd.GeoDataFrame(df, crs='EPSG:4326', geometry=geometry)
    return df

traffic_location_df.columns = ["NRA","siteId","Direction","Location","Lat","Lon","Nothing"]
traffic_location_df.drop(columns=["Nothing","NRA"],inplace=True)
traffic_location_df = traffic_location_df[~traffic_location_df["Location"].str.lower().str.contains("test")]
traffic_location_df.dropna(inplace=True)
traffic_location_df.reset_index(inplace=True,drop=True)
traffic_location_df["Lat"]=traffic_location_df["Lat"].astype(float)
traffic_location_df["Lon"]=traffic_location_df["Lon"].astype(float)
traffic_location_df = generate_geometry(traffic_location_df,"Lat","Lon")
traffic_location_df = traffic_location_df.drop_duplicates(subset=["geometry"])
traffic_location_df.reset_index(drop=True,inplace=True)
traffic_location_df["siteId"] =traffic_location_df["siteId"].str.strip("0")


In [ ]:
# print(tabulate(traffic_location_df, headers='keys', tablefmt='psql'))
# type(traffic_location_df["Lat"][0])

### Clean Road Network

In [ ]:
def find_closet_traffic_counter(row,gpd_network):
    gpd_network["Distance"]=gpd_network["geometry"].apply(lambda x: x.distance(row))
    min_id = gpd_network[gpd_network["Distance"]==min(gpd_network["Distance"].unique().tolist())]["OBJECTID"].tolist()
    return min_id[0]
traffic_location_df["Road_ID"]=traffic_location_df["geometry"].apply(lambda x: find_closet_traffic_counter(x,gpd_network))
gpd_network.drop(columns=["Distance"],inplace=True)
gpd_network.reset_index(drop=True,inplace=True)
selected_route = gpd_network[gpd_network["OBJECTID"].isin(traffic_location_df["Road_ID"].unique())].reset_index(drop=True)

traffic_location_df = traffic_location_df.merge(traffic_count_df,how="inner",on="siteId")
traffic_location_df['colour'] = traffic_location_df['traffic_count'].map(cmap)
# traffic_location_df = pd.merge(traffic_location_df,selected_route,how="inner",left_on="Road_ID",right_on="OBJECTID")

### Generate Map with Folium

### Slider for Linestring

In [ ]:
# temp = traffic_location_df.drop(columns=["Direction","Lat","Lon","siteId"])

In [ ]:
# style_dict2 = {}
# road_ID_list = temp['Road_ID'].unique().tolist()
# temp["DATE"]=(pd.DatetimeIndex(temp['Time']).astype(int)// 10 ** 9).astype('U10')
# road_idx = range(len(road_ID_list))
# for i in road_idx:
#     road = road_ID_list[i]
#     result = temp[temp['Road_ID'] == road]
#     inner_dict = {}
#     for _, r in result.iterrows():
#         inner_dict[r['DATE']] = {'color': r['colour'], 'opacity': 1}
#     style_dict2[i] = inner_dict


In [ ]:
def create_geojson_features(df):
    features = []
    for _, row in df.iterrows():
        feature = {
            'type': 'Feature',
            'geometry': {
                'type':'Point', 
                'coordinates':[row['Lon'],row['Lat']]
            },
            'properties': {
                'time': row['Time'],
                'style': {'color' : row['colour']},
                'icon': 'circle',
                'iconstyle':{
                    'fillColor': row['colour'],
                    'fillOpacity': 0.8,
                    'stroke': 'true',
                    'radius': 4
                }
            }
        }
        features.append(feature)
    return features

In [ ]:
def make_map(features):
    center = [53.52876390714696, -8.070723942287891]
    bounds = [[51.381808536222344, -4.9184898320881025],[55.45217379512414, -11.451803787292237]]
    road_count_map = folium.Map(location = center, zoom_start=7, tiles='cartodbpositron',min_zoom = 7, maxBounds= bounds,
      maxBoundsViscosity= 1,control_scale=True)
    
    TimestampedGeoJson(
        {'type': 'FeatureCollection',
        'features': features}
        , period='P7D'
        , add_last_point=True
        , auto_play=False
        , loop=False
        , max_speed=1
        , transition_time=1
        , loop_button=True
        , date_options='DD/MM/YYYY'
        , time_slider_drag_update=True
    ).add_to(road_count_map)
    return road_count_map

def plot_pollutant(df):
    features = create_geojson_features(df)
    return make_map(features), df

In [ ]:
road_count_map, df = plot_pollutant(traffic_location_df)

In [ ]:
max_colour2 = max(traffic_location_df['traffic_count'])
min_colour2 = sorted(traffic_location_df['traffic_count'].unique().tolist())[1]
defined_scale2 = [i for i in range(int(min_colour), int(max_colour2)+int(max_colour2//12), int(max_colour2//12))]
cmap2 = cm.linear.RdYlBu_11.to_step(12,index=defined_scale2)
cmap2.caption = "Average numbers of movement every week"
road_count_map.add_child(cmap2)


lea_layer = folium.FeatureGroup(name="LEAs",show=True).add_to(road_count_map)
style2 = {'color': '#00FFFFFF'}
lea_layer.add_child(folium.GeoJson(data=short_gpd,popup=folium.GeoJsonPopup(fields=["LE_ID"]),style_function=lambda x:style2)).add_to(road_count_map)


lea_layer = folium.FeatureGroup(name="Road Network",show=True).add_to(road_count_map)
lea_layer.add_child(folium.GeoJson(data=selected_route)).add_to(road_count_map)


folium.LayerControl(position="bottomright").add_to(road_count_map)

road_count_map.fit_bounds(center) 
road_count_map.save("RoadNetwork"+".html")
# road_count_map

In [ ]:
from IPython.core.display import display, HTML

htmlmap = HTML('<iframe srcdoc="{}" style="float:left; width: {}px; height: {}px; display:inline-block; width: 50%; margin: 0 auto; border: 2px solid black"></iframe>'
           '<iframe srcdoc="{}" style="float:right; width: {}px; height: {}px; display:inline-block; width: 50%; margin: 0 auto; border: 2px solid black"></iframe>'
           .format(slider_map.get_root().render().replace('"', '&quot;'),1000,1000,
                   road_count_map.get_root().render().replace('"', '&quot;'),1000,1000))
display(htmlmap)